# **Deforum Colab Notebook (v0.9)**

## Setup

In [ ]:
#@markdown **Environment Setup**
import os
import subprocess
import sys
import time

def setup_environment():
    # Check if in google colab
    try:
        ipy = get_ipython()
        if 'google.colab' not in str(ipy):
            print('Not running on Google Colab. Environment setup aborted.\n')
            print('```')
            print('Environment: Non Google Colab')
            print('Error: This notebook is not running in Google Colab.')
            print('```')
            print("Please ensure this notebook runs in Google Colab.")
            print("If this problem persists, please open an issue at: https://github.com/deforum-studio/deforum/issues")
            return
    except Exception:
        print("Error! Not running in an IPython environment. Environment setup aborted.\n")
        print('```')
        print('Environment: Non IPython')
        print('Error: This notebook is not running in an IPython environment.')
        print('```')
        print("If this problem persists, please open an issue at: https://github.com/deforum-studio/deforum/issues")
        return

    # Start the timer
    start_time = time.time()

    # Specify requirements in a list for easy editing
    requirements = ['einops', 'timm', 'basicsr', 'accelerate', 'transformers', 'torchsde']

    try:
        # Clone the deforum repo
        process = subprocess.Popen(['git', 'clone', 'https://github.com/deforum-studio/deforum.git'])
        process.wait()

        # Install required packages in one go
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', *requirements])

        # Append the path
        sys.path.append('./deforum/src/')
    except Exception as e:
        print("An error occurred during the setup.\n")
        print('```')
        print('Error:', str(e))
        print('```')
        print("Please copy the above text and open an issue at: https://github.com/deforum-studio/deforum/issues")
        return

    # Report the total time
    total_time = time.time() - start_time
    print('Environment setup completed successfully. Total time: {:.2f} seconds'.format(total_time))

setup_environment()

In [ ]:
#@markdown **Path Setup**
def path_setup():
    # Start the timer
    try:
        start_time = time.time()
    except:
        print("Rerun the previous cell")
    try:
        models_path = "models" #@param {type:"string"}
        output_path = "output" #@param {type:"string"}
        mount_google_drive = True #@param {type:"boolean"}
        force_remount = False
        models_path_gdrive = "/content/drive/MyDrive/AI/models" #@param {type:"string"}
        output_path_gdrive = "/content/drive/MyDrive/AI/output" #@param {type:"string"}

        print("Setting up paths...")

        try:
            ipy = get_ipython()

            if 'google.colab' in str(ipy):
                print("Executing in Google Colab...")

                if mount_google_drive:
                    print("Mounting Google Drive...")

                    from google.colab import drive # type: ignore
                    try:
                        drive_path = "/content/drive"
                        drive.mount(drive_path,force_remount=force_remount)
                        models_path = models_path_gdrive
                        output_path = output_path_gdrive

                        print("Mounted Google Drive successfully.")
                    except:
                        print("Error mounting drive or with drive path variables.")
                        print("Reverting to default path variables.")

        except:
            print("Executing in a local environment...")
            print("Using default path variables.")

    except Exception as e:
        print(f"An unexpected error occurred during the path setup: {str(e)}")
        print("If you encounter any issues, please copy this console output and open an issue at: https://github.com/deforum-studio/deforum/issues")

    finally:
        print(f"Models will be saved in this path: {models_path}")
        print(f"Output will be saved in this path: {output_path}")

    # Report the total time
    total_time = time.time() - start_time
    print('Path setup completed successfully. Total time: {:.2f} seconds'.format(total_time))
    return models_path, output_path

models_path, output_path = path_setup()

In [ ]:
#@markdown **Model Setup**
import os, re
from urllib.parse import urlparse

# import deforum
try:
    from deforum import DeforumAnimationPipeline
except Exception as e:
    print("An error occurred during the import.")
    print('```')
    print('Error:', str(e))
    print('```')
    print("Please copy the above text and open an issue at: https://github.com/deforum-studio/deforum/issues")


def get_civitai_model_id(civitai_url):
    try:
        model_id = re.search(r"models/(\d+)", civitai_url).group(1)
        if not model_id:
            print("No Model ID found in the download links.")
            return None

        print(f"Model ID successfully obtained: {model_id}")
        return model_id

    except Exception as e:
        print(f"An unexpected error occurred during the model setup: {str(e)}")
        print("If you encounter any issues, please copy this console output and open an issue at: https://github.com/deforum-studio/deforum/issues")


def instantiate_deforum_pipeline(models_path):

    # Start the timer
    start_time = time.time()

    model_map = {
        "SDXL-Base v1.0": {"url": "https://civitai.com/models/101055?modelVersionId=128078"},
        "SDXL-ProtoVisonXL_0630": {"url": "https://civitai.com/models/125703?modelVersionId=201514"},
        "SDXL-DynaVisionXL_0557": {"url": "https://civitai.com/models/122606?modelVersionId=198962"},
        "SDXL-NightVisionXL_0770": {"url": "https://civitai.com/models/128607?modelVersionId=201643"},
        "SDXL-ZavyChromaXL_v2.1": {"url": "https://civitai.com/models/119229?modelVersionId=169740"},
        "SDXL-JuggernautXL_v6": {"url": "https://civitai.com/models/133005?modelVersionId=198530"},
        "SDXL-UnstableDiffusers_YamerMix_v8": {"url": "https://civitai.com/models/84040?modelVersionId=196039"},
        "SDXL-Copax_TimelessXL_v8": {"url": "https://civitai.com/models/118111?modelVersionId=198246"},
        "SD_1.5-RealisticVision_v5.1": {"url": "https://civitai.com/models/4201?modelVersionId=130072"},
        "SD_1.5-Dreamshaper_v8": {"url": "https://civitai.com/models/4384/dreamshaper?modelVersionId=128713"},
        "SD_1.5-RevAnimated_v.1.2.2": {"url": "https://civitai.com/models/7371?modelVersionId=46846"},
        "SD_1.5-GhostMix_v2": {"url": "https://civitai.com/models/36520?modelVersionId=76907"}
    }

    try:
        model_checkpoint =  "SDXL-ProtoVisonXL_0630" #@param ['(model_path_or_url)', 'SDXL-Base v1.0', 'SDXL-ProtoVisonXL_0630', 'SDXL-DynaVisionXL_0557', 'SDXL-NightVisionXL_0770', 'SDXL-ZavyChromaXL_v2.1', 'SDXL-JuggernautXL_v6', 'SDXL-UnstableDiffusers_YamerMix_v8', 'SDXL-Copax_TimelessXL_v8', 'SD_1.5-RealisticVision_v5.1', 'SD_1.5-Dreamshaper_v8', 'SD_1.5-RevAnimated_v.1.2.2', 'SD_1.5-GhostMix_v2']
        model_path_or_url = '' #@param {type:"string"}
        force_download = False #@param {type:"boolean"}

        if model_checkpoint == "(model_path_or_url)":
            if os.path.isfile(model_path_or_url):
                print("Loading local model file.\n")
                deforum = DeforumAnimationPipeline.from_file(model_path=model_path_or_url)
            else:
                parse_result = urlparse(model_path_or_url)
                if parse_result.scheme and parse_result.netloc:
                    print("Loading model from URL.\n")
                    model_id = get_civitai_model_id(model_path_or_url)
                    try:
                        deforum = DeforumAnimationPipeline.from_civitai(model_id=str(model_id), cache_dir=models_path, force_download=force_download)
                    except Exception as e:
                        print(f"Error loading model with from_civitai ({str(e)}), trying from_download_url.\n")
                        deforum = DeforumAnimationPipeline.from_url(model_path_or_url, cache_dir=models_path, force_download=force_download)
                else:
                    print(f"model_checkpoint: {model_checkpoint}")
                    print(f"model_path_or_url: {model_path_or_url}")
                    print("The model_path_or_url you provided is neither a valid local file nor a URL.\nPlease try again or select a model from the dropdown.")
                    print("If you believe this is an bug, please copy this console output and open an issue at: https://github.com/deforum-studio/deforum/issues")
        else:
            model_path_or_url = model_map[model_checkpoint]["url"]
            model_id = get_civitai_model_id(model_path_or_url)
            try:
                deforum = DeforumAnimationPipeline.from_civitai(model_id=str(model_id), cache_dir=models_path, force_download=force_download)
            except Exception as e:
                print(f"Error loading model with from_civitai ({str(e)})")
                print(f"Try checking `force_download` and try again.")
                print("If you believe this is an bug, please copy this console output and open an issue at: https://github.com/deforum-studio/deforum/issues")

    except Exception as e:
        print(f"An unexpected error occurred during the model setup: {str(e)}")
        print("If you encounter any issues, please copy this console output and open an issue at: https://github.com/deforum-studio/deforum/issues")

    # Report the total time
    total_time = time.time() - start_time
    print('Model setup completed successfully. Total time: {:.2f} seconds'.format(total_time))
    return deforum

pipeline = instantiate_deforum_pipeline(models_path)

## Run

In [ ]:
from IPython.display import display, Image

def display_image_from_dict(image_dict):
    if 'image' in image_dict:
        image_data = image_dict['image']
        # If the image data is a path to an image file
        if isinstance(image_data, str):
            display(Image(filename=image_data))
        # If the image data is in bytes or a PIL image object
        else:
            display(Image(data=image_data))
    else:
        raise ValueError("The dictionary does not contain an 'image' key.")

def DeforumArgs():

    prompt = ""
    style = "(No Style)"
    keyframes = "0"
    max_frames = 1
    W = 768
    H = 768
    scale = 7.0
    steps = 10
    sampler_name = "euler"
    scheduler = "normal"
    sampling_mode = "bicubic"
    seed = 1892049175
    seed_behavior = "iter"
    angle = "0:(0)"
    zoom = "0:(1.00)"
    translation_x = "0:(0)"
    translation_y = "0:(0)"
    translation_z = "0:(0)"
    rotation_3d_x = "0:(0)"
    rotation_3d_y = "0:(0)"
    rotation_3d_z = "0:(0)"
    noise_schedule = "0: (0.02)"
    strength_schedule = "0: (0.45)"
    contrast_schedule = "0: (1.0)"
    flip_2d_perspective = False
    perspective_flip_theta = "0:(0)"
    perspective_flip_phi = "0:(0)"
    perspective_flip_gamma = "0:(0)"
    perspective_flip_fv = "0:(53)"
    ddim_eta = 0.0
    make_grid = False
    grid_rows = 2
    use_mask = False
    use_alpha_as_mask = False
    mask_file = "./scripts/deforum/init_files/mask.jpg"
    invert_mask = False
    mask_brightness_adjust = 1.0
    mask_contrast_adjust = 1.0
    use_init = False
    strength = 0.0
    strength_0_no_init = True
    init_image = "./scripts/deforum/init_files/init.jpg"
    video_init_path = "False"
    hybrid_comp_alpha_schedule = "0:(0.5)"
    hybrid_comp_mask_blend_alpha_schedule = "0:(0.5)"
    hybrid_comp_mask_contrast_schedule = "0:(1)"
    hybrid_comp_mask_auto_contrast_cutoff_high_schedule = "0:(100)"
    hybrid_comp_mask_auto_contrast_cutoff_low_schedule = "0:(0)"
    hybrid_flow_factor_schedule = "0:(1)"
    hybrid_generate_inputframes = False
    hybrid_generate_human_masks = "None"
    hybrid_use_first_frame_as_init_image = True
    hybrid_motion = "None"
    hybrid_motion_use_prev_img = False
    hybrid_flow_consistency = False
    hybrid_consistency_blur = 2
    hybrid_flow_method = "RAFT"
    hybrid_composite = "None"
    hybrid_use_init_image = False
    hybrid_comp_mask_type = "None"
    hybrid_comp_mask_inverse = False
    hybrid_comp_mask_equalize = "None"
    hybrid_comp_mask_auto_contrast = False
    hybrid_comp_save_extra_frames = False
    prompt_tmp = ""
    save_samples = True
    save_settings = False
    display_samples = True
    pathmode = "root"
    filename_format = "{timestring}_{index}_{seed}.png"
    frame_interpolation_engine = "FILM"
    frame_interpolation_x_amount = 2
    animation_mode = "None"
    border = "wrap"
    color_coherence = "Match Frame 0 LAB"
    turbo_steps = 1
    optical_flow_cadence = "None"
    use_depth_warping = True
    midas_weight = 0.3
    near_plane = 200
    far_plane = 10000
    fov = 40
    padding_mode = "border"
    save_depth_maps = False
    extract_nth_frame = 1
    interpolate_key_frames = False
    interpolate_x_frames = 4
    resume_from_timestring = False
    resume_timestring = "20220829210106"
    iterations = 1
    batch_size = 1
    animation_prompts = {"0": ""}
    diffusion_cadence = 1
    tiling = False
    restore_faces = False
    seed_resize_from_w = 1024
    seed_resize_from_h = 1024
    sampler = "euler_a"
    batch_name = "deforum_20231101211202"
    seed_iter_N = 1
    overlay_mask = True
    mask_overlay_blur = 4
    fill = "fill"
    full_res_mask = True
    full_res_mask_padding = 4
    reroll_blank_frames = "ignore"
    reroll_patience = 10
    engine = "internal"
    hybrid_use_full_video = True
    animation_prompts_positive = ""
    animation_prompts_negative = "nsfw, nude, ugly"
    transform_center_x = "0: (0.5)"
    transform_center_y = "0: (0.5)"
    enable_perspective_flip = False
    cfg_scale_schedule = "0: (6)"
    enable_steps_scheduling = False
    steps_schedule = "0: (25)"
    fov_schedule = "0: (70)"
    aspect_ratio_schedule = "0: (1)"
    aspect_ratio_use_old_formula = False
    near_schedule = "0: (200)"
    far_schedule = "0: (10000)"
    seed_schedule = "0:(-1)"
    pix2pix_img_cfg_scale_schedule = "0:(1.5)"
    enable_subseed_scheduling = False
    subseed_schedule = "0: (1)"
    subseed_strength_schedule = "0: (0)"
    enable_sampler_scheduling = False
    sampler_schedule = "0: (\"Euler a\")"
    use_noise_mask = False
    mask_schedule = "0: (\"{video_mask}\")"
    noise_mask_schedule = "0: (\"{video_mask}\")"
    enable_checkpoint_scheduling = False
    checkpoint_schedule = "0: (\"model1.ckpt\"), 100: (\"model2.safetensors\")"
    enable_clipskip_scheduling = False
    clipskip_schedule = "0: (2)"
    enable_noise_multiplier_scheduling = True
    noise_multiplier_schedule = "0: (1.05)"
    enable_ddim_eta_scheduling = False
    ddim_eta_schedule = "0: (0)"
    enable_ancestral_eta_scheduling = False
    ancestral_eta_schedule = "0: (1)"
    amount_schedule = "0: (0.1)"
    kernel_schedule = "0: (5)"
    sigma_schedule = "0: (1)"
    threshold_schedule = "0: (0)"
    color_coherence_image_path = ""
    color_coherence_video_every_N_frames = 1
    color_force_grayscale = False
    legacy_colormatch = False
    cadence_flow_factor_schedule = "0: (1)"
    optical_flow_redo_generation = "None"
    redo_flow_factor_schedule = "0: (1)"
    diffusion_redo = 0
    noise_type = "perlin"
    perlin_w = 8
    perlin_h = 8
    perlin_octaves = 4
    perlin_persistence = 0.5
    depth_algorithm = "Midas-3-Hybrid"
    extract_from_frame = 0
    extract_to_frame = -1
    overwrite_extracted_frames = False
    use_mask_video = False
    video_mask_path = "https://deforum.github.io/a1/VM1.mp4"
    parseq_use_deltas = True
    use_looper = False
    init_images = ""
    image_strength_schedule = "0:(0.75)"
    blendFactorMax = "0:(0.35)"
    blendFactorSlope = "0:(0.25)"
    tweening_frames_schedule = "0:(20)"
    color_correction_factor = "0:(0.075)"
    skip_video_creation = False
    fps = 15
    make_gif = False
    delete_imgs = False
    image_path = "C:/SD/20230124234916_%09d.png"
    add_soundtrack = "None"
    soundtrack_path = "https://deforum.github.io/a1/A1.mp3"
    r_upscale_video = False
    r_upscale_factor = "2x"
    r_upscale_model = "realesr-animevideov3"
    r_upscale_keep_imgs = True
    frame_interpolation_slow_mo_enabled = False
    frame_interpolation_slow_mo_amount = 2
    frame_interpolation_keep_imgs = False
    device = "cuda"
    models_path = "models/other"
    half_precision = True
    mask_preset_names = ["everywhere","video_mask"]
    frames_cache = []
    raw_batch_name = "deforum_20231101211202"
    raw_seed = 346402409820051196
    timestring = "20231101211202"
    subseed = -1
    subseed_strength = 0
    seed_internal = 0
    current_user_os = "Linux"
    tmp_deforum_run_duplicated_folder = "/tmp/tmp_run_deforum"
    outdir = "output/deforum/deforum_20231101211202"
    start_frame = 0
    frame_idx = 0
    turbo_prev_frame_idx = 0
    turbo_next_frame_idx = 0
    contrast = 1.0
    use_areas = False
    prompt_series = {
        "0": "",
        "1": ""
    }
    using_vid_init = False
    predict_depths = True
    keep_in_vram = True

    return locals()

pipeline(**DeforumArgs())